In [ ]:
'''
Free Variables and closures 

Remember: Functions defined inside another function can access the outer (nonlocal) variables 

def outer(): 
    x = 'python'
    
    def inner(): 
        print('{0} rocks!'.format(x))  This x refers to the one in outer's scope/ This nonlocal variable x i scalled a free vraible 
        
    
    inner() 
    
outer()   -------> Python Rocks! 

When we consider inner, we realy are looking at: 
The fnction inner 
The free variable x (with current value python) 

This is called a closure 


Returning the inner function 

What happens if, instead of calling (running) inner from inside outer, we return it? 

def outer(): 
    x = 'python' 
    def inner() 
        prit('{0} rocks!'.format(x)) -----> x is a free variable in inner. It is bound to the variale x in outer. 
                                            This happens when outer runs (i.e. when inner is created) 
                                            This is the closure 
    inner()
    return inner                     -----> When we return inner, we are actually 'returning' the closure 

We can assign that return value to a vraible name: fn = outer() 

fn() ---> python rocks! 

when we called fn 
at the timme Python determined the vaue of x in the etxended scope. 

When we call fn, the outer has finished running before we called fn. The outer scope is gone. 
But how can x still be retained? 

Python Cells and Multi-Scoped Variables 

Two different scopes shared the same variable as shown in the last example 

def outer():                      Here is the value of x is shared between two scopes: 
    x = 'python'                     1. outer
    def inner():                     2. closure
        print(x)                  The label is in two different scopes but always reference the name 'value' 
    return inner 

Python does this by creating a cell as an itermediary object. 
The first x creates A cell ---> memory address and a referent to another object: string 'python' stored at a different memory address

Outer x
inner x    both points to the cell (intermediary) ---> then to string 'python' (indirect reference)

In effect, both variables x (in outer and inner), point to the same cell. 
When requesting the value of the variable, Python will 'double-hop' to get to the final value. 



Closure 

You can think of the closure as a function plus an extended scope that contains the free variables. 

The free variable's is the object the cell points to - so that could change over time! 

Every time the function is the closure is called and the free vairable is referenced: 

Python looks up the cell object, and then whatever the cell is pointing to 

def outer(): 
    a = 100 
    
    x = 'python' 
    def inner(): 
        a = 10 # local variable 
        print('{0} rocks!'.format(x))        ----> This section is the course   CELL (x) ---> Str 'python'
        
        
    return inner 
fn = outer() fn -> inner + extended scope 


Introspection 

def outer(): 
    a = 100 
    x = 'python' 
    def inner(): 
        a = 10 # local variable 
        print('{0} rocks!'.format(x)) 
    return inner 

fn = outer() 

fn.___code__.co_freevars ---> ('x',) (a is not a free variable) 

fn.__closure__   ----> (<cell at 0xA500: str object at 0xFF10>,) 

def outer(): 

    x = 'python' 
    print(hex(id(x)))  ------> 0xFF100 indirect reference 
    def inner(): 
        print(hex(id(x))) ------> 0xFF100 indirect reference 
        print('{0} rocks!'.format(x)) 
    return inner 
    
fn = outer() 
fn()


Modifying free variables 

def couter(): 

    count = 0
    def inc(): 
        nonlocal count         -----> count is a free variable. It is cound to the cell count 
        count += 1
        return count 
        
    return inc 

fn = counter()          ------> fn ---> int + cout --> 0

fn()  --> 1 count's (indirect) reference chnaged from the object 0 to the object 1 
fn()  --> 2 


Multiple Instances of Closures 
Every time we runa function, a new scope is created. 
If that function generates a closure, a new closure is created every time as well. 

def counter():                f1 = counter()  ----> a local scope
    count = 0                 f2 = counter()  ----> a new local scope   ----> two different closures 
    
    def inc(): 
        nonlocal count
        count += 1 
        return count 
        
    return inc 
    
f1() -->1
f1() -->2
f1() -->3
f1() -->4


f2() --> 1    f1 and f2 do not have the same extended scope. They are different instances of the closure. 
              The cells are different. 

Shared Extended Scopes 

def outer(): 
    
    count = 0 
    
    def inc1(): 
        nonlocal count        count is a free variable - bound to count in the extended scope
        count += 1 
        return count 
    
    def inc2(): 
        nonlocal count        count is a free variable - bound to the same count 
        count += 1 
        return count 

    return inc1, inc2         returns a tuple containing both closures 

f1, f2 = outer() 
f1() -> 1
f2() 


Shared Extended Scopes 
You may think this shared extended scope is highly unusal but it is not!! 
Usually by mistake 

def adder(n): 
    def inner(x):       inner and n are closure. both n point t the same place
       return x + n

    return inner 
    
add_1 = adder(1) 
add_2 = adder(2) 
add_3 = adder(3)        Three different closures  - no shared scopes 

add_1(10)   -> 11
add_2(10)   -> 12
add_3(10)   --> 13

Shared Extended Scopes 

But suppose we tried doing it this way: 

address = []
for n in range(1,4): 
    address.append(lambda x: x+n) 
    
n =1 : three variable in the lambda is n, and it is bound to the n we created in the loop 
n =2 : three variable in the lambda is n, and it is bound to the (same) n we created in the loop 
n =1 : three variable in the lambda is n, and it is bound to the (same) n we created in the loop 

Now we could call the address in the following way: 

adders[0](10)  ---> 13
adders[1](10)  ---> 13
adders[2](10)  ---> 13

Remember, Python does not 'evaluate' the free variable n until the address[i] fuction is called. 

Since all three functions in adders are bound to the same n by the time we call adders[0], the value of n is 3
(the last iteration of the loop set n to 3)

Nested Closures 

def incrementer(n): 
    # inner + n is a closure 
    def inner(start): 
        current = start 
        # inc + current + n is a closure 
        def inc(): 
            nonlocal current 
            current += n 
            return current 
        return inc 
    return inner 
    
    
(inner) 
fn = incremener(2)  --> fn.__code__.co_freevars -> 'n' n = 2
(inc)
inc_2 = fn (100)  --> inc_2.__code__.co_freevars --> 'current', 'n'
current = 100, n = 2

(calls inc)
inc_c()  -->102 (current = 102, n=2)
inc_c()  -->104 (current = 104, n=2)

# Nested closure is common 


'''